# Persistence

Many AI applications need memory to share context across multiple interactions. In LangGraph, memory is provided for any [StateGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.StateGraph) through [Checkpointers](https://langchain-ai.github.io/langgraph/reference/checkpoints/).

When creating any LangGraph workflow, you can set them up to persist their state by doing using the following:

1. A [Checkpointer](https://langchain-ai.github.io/langgraphjs/reference/classes/index.BaseCheckpointSaver.html), such as the [MemorySaver](https://langchain-ai.github.io/langgraphjs/reference/classes/index.MemorySaver.html)
2. Call `compile(checkpointer=myCheckpointer)` when compiling the graph.

<!-- Example:
```javascript
from langgraph.graph import StateGraph
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

builder = StateGraph(....)
# ... define the graph
memory = AsyncSqliteSaver.from_conn_string(":memory:")
graph = builder.compile(checkpointer=memory)
...
``` -->

This works for [StateGraph](https://langchain-ai.github.io/langgraphjs/reference/classes/index.StateGraph.html and all its subclasses, such as [MessageGraph](https://langchain-ai.github.io/langgraph/reference/graphs/#messagegraph).

Below is an example.

<div class="admonition tip">
    <p class="admonition-title">Note</p>
    <p>
        In this how-to, we will create our agent from scratch to be transparent (but verbose). You can accomplish similar functionality using the <code>create_react_agent(model, tools=tool, checkpointer=checkpointer)</code> (<a href="https://langchain-ai.github.io/langgraphjs/reference/functions/prebuilt.createReactAgent.html">API doc</a>) constructor. This may be more appropriate if you are used to LangChain’s <a href="https://python.langchain.com/v0.1/docs/modules/agents/concepts/#agentexecutor">AgentExecutor</a> class.
    </p>
</div>


## Setup

This guide will use Anthropic's Claude model. We will optionally set our API key for [LangSmith tracing](https://smith.langchain.com/), which will give us best-in-class observability.

In [ ]:
// Deno.env.set("ANTHROPIC_API_KEY", "sk_...");

// Optional, add tracing in LangSmith
// Deno.env.set("LANGCHAIN_API_KEY", "sk_...");
// Deno.env.set("LANGCHAIN_TRACING_V2", "true");
// Deno.env.set("LANGCHAIN_PROJECT", "Persistence: LangGraphJS");

## Define the state

The state is the interface for all of the nodes in our graph.

In [9]:
import { BaseMessage } from "@langchain/core/messages";

interface AgentStateChannels {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => BaseMessage[];
    default: () => BaseMessage[];
  };
  next: string;
}

// This defines the agent state
const agentStateChannels: AgentStateChannels = {
  messages: {
    value: (x: BaseMessage[], y: BaseMessage[]) => x.concat(y),
    default: () => [],
  },
};

## Set up the tools

We will first define the tools we want to use. For this simple example, we will use create a placeholder search engine.
However, it is really easy to create your own tools - see documentation [here](https://js.langchain.com/v0.2/docs/how_to/custom_tools) on how to do that.

In [12]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";

const searchTool = new DynamicStructuredTool({
  name: "search",
  description: "Use to surf the web",
  schema: z.object({
    query: z.string().describe("The query to use in your search."),
  }),
  func: async ({ query }: { query: string }) => {
    // This is a placeholder for the actual implementation
    return ["The answer to your question lies within."];
  },
});

await searchTool.invoke({ query: "What's the meaning of it all?" });

const tools = [searchTool]

[ "The answer to your question lies within." ]

We can now wrap these tools in a simple [ToolNode](https://langchain-ai.github.io/langgraphjs/reference/classes/prebuilt.ToolNode.html). This object will actually run the tools (functions) whenever they are invoked by our LLM.

In [29]:
import { ToolNode } from "../../langgraph/dist/prebuilt/index.cjs";
const toolNode = new ToolNode([tools]);

ReferenceError: exports is not defined

In [15]:
typeof ToolNode

"undefined"

## Set up the model

Now we will load the [chat model]().
https://js.langchain.com/v0.2/docs/concepts/#chat-models
1. It should work with messages. We will represent all agent state in the form of messages, so it needs to be able to work well with them.
2. It should work with tool calling. This means it should either be an OpenAI model or a model that exposes a similar interface.

Note: these model requirements are not requirements for using LangGraph - they are just requirements for this one example.